In [5]:

import warnings

import matplotlib.pyplot as plt
# import MultiTrade
import numpy as np
import pandas as pd
from IPython.display import display
from neuralforecast.core import NeuralForecast

from neuralforecast.auto import AutoBiTCN, AutoNBEATS, AutoTFT
from neuralforecast.losses.pytorch import RMSE

# from statsforecast import StatsForecast

from utils.utils import build_market_image,train_test_split_data,plot_insample_forecasts,refresh_crypto_data
from configs import defaults
import datetime
warnings.filterwarnings("ignore")

In [6]:
from configs import fx_defaults as defaults
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame
DATA_DIR=defaults.DATA_DIR
DATA_DIR

'data_forex'

In [7]:

horizon = 7
context_length=5*horizon
split_n_days=30
round_time='H'
id_col='symbol'
save_path=defaults.forecasting_model_path

In [8]:
# refresh_crypto_data(data_dir=DATA_DIR,time_frame=time_frame,pairs=COIN_PAIRS)
data=build_market_image(target_pair=target_pair,time_frame=time_frame,axis=0,data_dir=DATA_DIR,only_target=True)
# data.index

train_data,test_data=train_test_split_data(data=data,n_days=30)

# data['symbol'].unique()

# test_data
data

['data_forex/oanda-USDJPY-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 14194.54it/s]
1it [00:00,  6.51it/s]

2024-10-23 14:00:00 2024-11-22 14:00:00


,open,high,low,close,volume,symbol,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K,ds,unique_id
date_close,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,140.858,140.993,140.858,140.902,388,USDJPY,140.902,0.456522,-0.500000,-0.500000,-0.500000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-01-01 22:00:00,20
2024-01-01 23:00:00,140.902,141.132,140.811,141.116,3558,USDJPY,141.116,0.500000,-0.500000,-0.500000,-0.500000,0.000000,0.00000,0.000000,0.004801,0.002667,100.000000,0.000000,2024-01-01 23:00:00,20
2024-01-02 00:00:00,141.115,141.384,141.100,141.142,3790,USDJPY,141.142,-0.500000,-0.333333,-0.466667,-0.497260,0.000000,0.00000,0.000000,0.006924,0.004412,100.000000,0.000000,2024-01-02 00:00:00,20
2024-01-02 01:00:00,141.146,141.458,141.142,141.440,4419,USDJPY,141.440,-0.456522,-0.333333,-0.466667,-0.497260,0.000000,0.00000,0.000000,0.018479,0.009177,100.000000,0.000000,2024-01-02 01:00:00,20
2024-01-02 02:00:00,141.441,141.673,141.366,141.565,3846,USDJPY,141.565,-0.413043,-0.333333,-0.466667,-0.497260,0.000000,0.00000,0.000000,0.029343,0.015176,100.000000,0.000000,2024-01-02 02:00:00,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22 09:00:00,154.512,154.824,154.250,154.500,31886,USDJPY,154.500,-0.108696,0.166667,0.200000,0.393151,154.856859,154.47485,154.092841,-0.040257,-0.083803,47.626806,53.892821,2024-11-22 09:00:00,20
2024-11-22 10:00:00,154.500,154.555,154.252,154.328,20136,USDJPY,154.328,-0.065217,0.166667,0.200000,0.393151,154.833154,154.48625,154.139346,-0.054467,-0.077936,43.619669,36.501517,2024-11-22 10:00:00,20
2024-11-22 11:00:00,154.328,154.470,154.278,154.316,14633,USDJPY,154.316,-0.021739,0.166667,0.200000,0.393151,154.833462,154.48595,154.138438,-0.065937,-0.075536,43.345660,35.288170,2024-11-22 11:00:00,20


In [9]:
front=['y','ds','symbol']
exo_gen_cols=data.filter(like='feature_').columns.tolist()
cols=front+[c for c in exo_gen_cols if c not in front]

train_data=train_data[cols]
test_data=test_data[cols]
# train_data
# data[id_col].unique()
# 

In [10]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [11]:
forecast_horizon=6
backend='optuna'
BiTCN_config=AutoBiTCN.get_default_config(h=forecast_horizon,backend=backend)

TFT_config=AutoTFT.get_default_config(h=forecast_horizon,backend=backend)
NBEATS_config=AutoNBEATS.get_default_config(h=forecast_horizon,backend=backend)

In [12]:
# BiTCN_MODEL= AutoBiTCN(h=forecast_horizon,
#                   loss=RMSE(),
#                   config=BiTCN_config,
#                   search_alg=optuna.samplers.TPESampler(),
#                   backend='optuna',
#                   num_samples=10)

TFT_MODEL= AutoTFT(h=forecast_horizon,
                  loss=RMSE(),
                  config=TFT_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

NBEATS_MODEL= AutoNBEATS(h=forecast_horizon,
                  loss=RMSE(),
                  config=NBEATS_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

models=[TFT_MODEL,NBEATS_MODEL]

In [13]:

model = NeuralForecast(models=models,
                    local_scaler_type='robust',
                    
                     freq='1h')

In [14]:
model.fit(train_data,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        id_col=id_col,
        
        )


Seed set to 12
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | RMSE                     | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 256    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 154 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 64.8 K | train
6 | output_adapter          | Linear                   | 65     | train
-----------------------------------------------------------------------------
220 K     Trainable params
0         Non-trainable params
220 K     Total params
0.880     Total estimated model params size (MB)
88     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


In [15]:
model.save(save_path,overwrite=True)
save_path

'ForecastingModels/USDJPYForecastingModel/'

In [16]:
test_dates=test_data.sort_values('ds')['ds'].unique()
last_date=train_data['ds'].unique()[-1]

In [17]:
test_data

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
0,153.026,2024-10-23 14:00:00,USDJPY,0.108696,-0.166667,0.233333,0.310959,153.556723,152.06035,150.563977,0.558213,0.476774,89.909844,90.697674
1,152.820,2024-10-23 15:00:00,USDJPY,0.152174,-0.166667,0.233333,0.310959,153.609409,152.14545,150.681491,0.535647,0.488549,77.541344,76.144423
2,152.858,2024-10-23 16:00:00,USDJPY,0.195652,-0.166667,0.233333,0.310959,153.642395,152.23365,150.824905,0.514895,0.493818,78.138772,77.414966
3,152.776,2024-10-23 17:00:00,USDJPY,0.239130,-0.166667,0.233333,0.310959,153.631505,152.31995,151.008395,0.486227,0.492300,73.589594,71.585450
4,152.662,2024-10-23 18:00:00,USDJPY,0.282609,-0.166667,0.233333,0.310959,153.578197,152.39955,151.220903,0.449131,0.483666,67.689444,54.794521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,154.500,2024-11-22 09:00:00,USDJPY,-0.108696,0.166667,0.200000,0.393151,154.856859,154.47485,154.092841,-0.040257,-0.083803,47.626806,53.892821
523,154.328,2024-11-22 10:00:00,USDJPY,-0.065217,0.166667,0.200000,0.393151,154.833154,154.48625,154.139346,-0.054467,-0.077936,43.619669,36.501517
524,154.316,2024-11-22 11:00:00,USDJPY,-0.021739,0.166667,0.200000,0.393151,154.833462,154.48595,154.138438,-0.065937,-0.075536,43.345660,35.288170
525,154.498,2024-11-22 12:00:00,USDJPY,0.021739,0.166667,0.200000,0.393151,154.816346,154.47805,154.139754,-0.059653,-0.072359,48.617624,53.690597
